In [ ]:
!pip install checklist


In [ ]:
#imports

%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import sys
import numpy as np
import pandas as pd


import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

#model
model_name = "Constien/xmlRoberta_all_lang"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# sentiment analysis is a general name in Huggingface to load the pipeline for text classification tasks.
# set device=-1 if you don't have a gpu
pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, framework="pt", device=0)

In [3]:
editor = checklist.editor.Editor()
editor.tg

In [4]:
suite = TestSuite()

In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# test dataset
engset=pd.read_csv('/content/drive/MyDrive/IM/VTPAN_csv/VTPAN-test.csv')


In [9]:
#predatory examples
pred_set=engset.loc[engset['label'] == 'predator']

In [ ]:
#non predatory examples
nopred_set=engset.loc[engset['label'] == 'non-predator']

In [10]:
pred_set['segment_tur'] = pred_set['segment'].str[:1500]

<ipython-input-10-c395317695a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_set['segment_tur'] = pred_set['segment'].str[:1500]


In [ ]:
#pd.set_option('display.max_colwidth', None)
pred_set['segment_tur'].str.len().max()

1500

In [ ]:
nopred_set['segment_tur'] = nopred_set['segment'].str[:1500]

<ipython-input-12-5065e54092ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nopred_set['segment_tur'] = nopred_set['segment'].str[:1500]


In [11]:
sentences = pred_set['segment_tur']

pdata = list(nlp.pipe(sentences))

In [104]:
sentences_nopred = nopred_set['segment_tur']

pdata_nopred = list(nlp.pipe(sentences_nopred))

In [12]:
sentences=list(sentences)
#sentences_nopred=list(sentences_nopred)

**Capability: neumeric reasoning**

In [106]:
import re
def change_age(x, *args, **kwargs):
    # Returns empty or a list of strings with age changed
    age =  ['25', '30', '35', '40', '45', '50']
    age_replace= ['12', '13', '14','15', '16', '17']

    ret = []
    for p in age:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in age_replace if p != p2])
    return ret


In [109]:
ret = Perturb.perturb(sentences_nopred, change_age)
#ret.sentences

In [110]:
test = MFT(ret.data, labels=1)
suite.add(test,  'Age', 'Neumeric Reasoning', overwrite=True )

In [111]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
suite.summary()

In [ ]:
suite.run(pred_and_conf, overwrite=True)

Running single positive words
Predicting 62 examples
Running Age
Predicting 5340 examples


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
suite.summary()  # label 1, complete data 1 and 0

Vocabulary

single positive words
Test cases:      62
Fails (rate):    0 (0.0%)


Age
Test cases:      498
Fails (rate):    475 (95.4%)

Example fails:
1.0 hi hey asl? asl? female 16 you? from? 17 male germany you? norway :) :) how are you? fine thank you :) you have a girlfriend? ;p i´m fine too but bored :/ no do you have a boyfriend? no, i have a girlfriend right now, but we are not that serious ;p lesbian? ;) no, i mostly like boys ;) now i´m a bit horny ;) haha, eally? x) really* a bit :P do you have msn? no, sorry :p any other messenger? :D no a webcam? :D
1.0 hi hey asl? asl? female 16 you? from? 18 male germany you? norway :) :) how are you? fine thank you :) you have a girlfriend? ;p i´m fine too but bored :/ no do you have a boyfriend? no, i have a girlfriend right now, but we are not that serious ;p lesbian? ;) no, i mostly like boys ;) now i´m a bit horny ;) haha, eally? x) really* a bit :P do you have msn? no, sorry :p any other messenger? :D no a webcam? :D
1.0 hi hey asl

**Capability: vocabulary**

In [ ]:
#parent nouns
#parent_noun = ['dad', 'mom', 'aunt', 'grandma', 'uncle', 'grandfather', 'my grandmother', 'grandpa', 'my older sister', 'brother']
#editor.add_lexicon('parent_noun', parent_noun, overwrite=True)

names = ['my sweety', 'baby', 'babyy', 'babbby', 'little girl', 'sexy', 'darling','cutey', 'pussycat', 'angel', 'kiddo', 'kid', 'my girl', 'gorgeous']
editor.add_lexicon('names', names, overwrite=True)

In [ ]:
# positive adjectives
pos_adj = ['sexy','mesexy', 'great', 'allsome', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'attractive', 'girly', 'awesome', 'perfect', 'fun', 'cool', 'adorable', 'smart', 'exciting', 'sweet', 'wonderful']
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)


In [ ]:
#negative_adj = ['lame', 'shitty', 'not worthy', 'unattractive', 'not sexy', 'not beautiful', 'not nice']
#editor.add_lexicon('neutral_adj', pos_adj, overwrite=True)


In [ ]:
# get model suggestion for possible neutral verbs
#print(', '.join(editor.suggest('I cant {a:mask} you today, {parent_noun} is home tonight.')[:40]))

to, with, for, tell, see, call, of, told, update, from, t, thank, write, tel, at, visit, -, show, called, hug, on, post, miss, w, help, hear, di, warn, near, without, leave, hold, msg, get, th, message, around, make, meet, bother


In [ ]:
#pos_verb_present = ['like', 'enjoy', 'appreciate', 'love', 'admire', 'value']
#neutral_verb_present = ['call', 'see', 'visit', 'write', 'be near', 'come over', 'invite', 'msg', 'message', 'text', 'go online', 'log in', 'hug', 'touch', 'kiss', 'lick', 'smell']
#pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
#neutral_verb_past = ['called', 'saw', 'visited', 'wrote', 'was near', 'came over', 'invited', 'messaged', 'texted', 'came online', 'logged in', 'hugged', 'touched', 'kissed', 'licked', 'smelled']

#editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
#editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
#editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
#editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
#editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
#editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

Individual words

In [ ]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
def add_NiWords(x):
    phrases = ['my sweety', 'baby', 'babyy', 'babbby', 'little girl', 'sexy', 'darling','cutie', 'angel', 'kiddo', 'kid', 'my girl', 'gorgeous']
    return ['%s %s' % (x, p) for p in phrases]

In [ ]:
monotonic_increasing = Expect.monotonic(label=1, increasing=False, tolerance=0.1)


In [ ]:
t = Perturb.perturb(sentences, add_NiWords)
test = DIR(**t, expect=monotonic_increasing)

In [ ]:
test.run(pred_and_conf)
test.summary()

Predicting 21990 examples
Test cases:      1466
After filtering: 73 (5.0%)
Fails (rate):    11 (15.1%)

Example fails:
0.6 hey hi cutie whats up nm u? just watching tv thats cool whatr u watchin? lol american idol hey if i get quiet it is cause the wife is in the room with me lol lol okay im bout to watch that too it just started are you going to get off line no brb ok k i was gettin somthin to eat yay its back on welcome back what you get me to eat lol pinaple peices i like pinapple me to so what you been up too just did hw u? that dog is so cute me just working and stuff oh thats cool just been a long day i cant wait for the weekend yeah whatr u gonna do? well i got to work most of it oh lol but have to go to a wrestling meet sat morning then drop the wife off at the airport early sunday then go bowling sunday night cool what you doing im suposed to go to the mall w/ my freind sat what mall i dunno yet my freinds mom is takin us lol ok maybe westland or 12 oaks oh ok what you going t

Add phrases

In [ ]:
positive = editor.template('I {pos_verb_present} you.').data
positive += editor.template('You are my {pos_adj}.').data
positive += ['I would like to {neutral_verb_present} if your {parent_nount} not home.']
def add_phrase_function(phrases):
    def pert(d):
        while d[-1].pos_ == 'PUNCT':
            d = d[:-1]
        d = d.text
        ret = [d + '. ' + x for x in phrases]
        idx = np.random.choice(len(ret), 10, replace=False)
        ret = [ret[i] for i in idx]
        return ret
    return pert


In [ ]:
t = Perturb.perturb(pdata, add_phrase_function(positive), nsamples=500)
test = MFT(t.data, labels=1)
description = 'Add very predator laden words to the end of sentences, expect probability of negative to go up (tolerance=0.1)'
suite.add(test, 'add positive phrases', 'Vocabulary', description, overwrite=True)


In [ ]:
suite.run(pred_and_conf, overwrite=True)

Running add positive phrases
Predicting 5500 examples


In [ ]:
suite.summary()

Vocabulary

add positive phrases
Test cases:      500
Fails (rate):    451 (90.2%)

Example fails:
1.0 hey hi asl how's it going? 69 m/f hogwarts how about you? fine? 15,musa 15m usa coolios i was just joking about being 69 btw, LOL what ah,,hoe about your true age do you have facebook. LOL im actually 14 and yeah hey? ahh,,female yeah, i am a girl hahaha! ah ok,,, k k k k k k k k k but, sadly i dont live in hogwarts kkkkk k k wer do you live in a house
1.0 hey hi asl how's it going? 69 m/f hogwarts how about you? fine? 15,musa 15m usa coolios i was just joking about being 69 btw, LOL what ah,,hoe about your true age do you have facebook. LOL im actually 14 and yeah hey? ahh,,female yeah, i am a girl hahaha! ah ok,,, k k k k k k k k k but, sadly i dont live in hogwarts kkkkk k k wer do you live in a house. You are my fun.
1.0 hey hi asl how's it going? 69 m/f hogwarts how about you? fine? 15,musa 15m usa coolios i was just joking about being 69 btw, LOL what ah,,hoe about your true age

**Capability: robustness**

adding irrelevant stuff before and after.


In [ ]:
import string
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
def random_url(n=6):
    return 'https://t.co/%s' % random_string(n)
def random_handle(n=6):
    return '@%s' % random_string(n)


def add_irrelevant(sentence):
    urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    irrelevant_before = ['@user '] + urls_and_handles
    irrelevant_after = urls_and_handles
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets


In [ ]:
t = Perturb.perturb(pdata, add_irrelevant, nsamples=500)
test = INV(t.data)
suite.add(test, 'add random urls and handles', 'Robustness', 'add randomly generated urls and handles to the start or end of sentence', overwrite=True)


punctuation, contractions, typos




In [ ]:
t = Perturb.perturb(pdata, Perturb.punctuation, nsamples=500)
test = INV(t.data)
suite.add(test, 'punctuation', 'Robustness', 'strip punctuation and / or add "."', overwrite=True)


In [ ]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=1)
test = INV(t.data)
suite.add(test, 'typos', 'Robustness', 'Add one typo to input by swapping two adjacent characters', overwrite=True )


In [ ]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=2)
test = INV(t.data)
suite.add(test, '2 typos', 'Robustness', 'Add two typos to input by swapping two adjacent characters twice', overwrite=True )


In [ ]:
t = Perturb.perturb(sentences, Perturb.contractions, nsamples=1000)
test = INV(t.data)
suite.add(test, 'contractions', 'Robustness', 'Contract or expand contractions, e.g. What is -> What\'s', overwrite=True)

In [ ]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
suite.run(pred_and_conf, overwrite=True)

Running add random urls and handles
Predicting 11000 examples
Running punctuation
Predicting 1026 examples
Running typos
Predicting 1000 examples
Running 2 typos
Predicting 1000 examples
Running contractions
Predicting 2213 examples


In [ ]:
suite.summary()

Robustness

add random urls and handles
Test cases:      500
Fails (rate):    47 (9.4%)

Example fails:
0.9 close ur eys and picture a mushroom pizza mmmmmmmmmmmmm :D mmmmmm hello, my gothic princess hello, my gothic prince guy Prince Chris thats hard 2 say fast lol I can imagine although I say that in the mirror every night befor I go to bed LOL! :)) u do NOT u say hey d00d zzzzzzzzzzzzzzzzzzzzzzzzzzzzz how did you know! ? cuz u said u go 2 bed erly but that still doesn't explain...... hey! what can you turn into a bat afly up here and spy on me? and fly* lol um no honest? u gettin redy 2 go home rite? or a sprir
0.5 close ur eys and picture a mushroom pizza mmmmmmmmmmmmm :D mmmmmm hello, my gothic princess hello, my gothic prince guy Prince Chris thats hard 2 say fast lol I can imagine although I say that in the mirror every night befor I go to bed LOL! :)) u do NOT u say hey d00d zzzzzzzzzzzzzzzzzzzzzzzzzzzzz how did you know! ? cuz u said u go 2 bed erly but that still doesn't expl

In [ ]:
#suite.visual_summary_table()

**Capability: negation**

In [ ]:
# First, let's find some positive and negative adjectives
', '.join(editor.suggest('This do not want to {a:mask} you.')[:30])

'hurt, to, with, upset, stop, on, provoke, of, for, offend, help, you, attack, abuse, end, insult, embarrass, up, exploit, about, affect, out, change, inconvenience, injure, bother, off, educate, harm, impact'

In [20]:
def pred_and_conf(data):
    raw_preds = pipe(data)
    preds = np.array([ int(p["label"][-1]) for p in raw_preds])
    pp = np.array([[p["score"], 1-p["score"]] if int(p["label"][-1]) == 'predator' else [1-p["score"], p["score"]] for p in raw_preds])
    return preds, pp

In [ ]:
#verb = ['meet', 'call', 'chat', 'talk', 'see', 'exploit']
#ret = editor.template('I do not want to {verb} you.', verb=verb, labels=0, save=True, nsamples=300)

In [23]:
t = Perturb.perturb(pdata, Perturb.add_negation)
#ret.data

In [17]:
test = INV(t.data, labels=1)
suite.add(test, 'simple negations', 'Negation', 'Negating predator statements should result in non-predator predictions', overwrite=True )


In [ ]:
suite.run(pred_and_conf)

Running simple negations
Predicting 2932 examples


In [ ]:
suite.summary()

Negation

simple negations
Test cases:      1466
Fails (rate):    1395 (95.2%)

Example fails:
1.0 hi :) sorry about today but i left my keys to my office here and could not get into it and get on the computer bummer missed you so much missed u to i was signed on all day in case u came on you are so nice, sorry i let you down no prob i understand so how was your day? borin my aunt was home again i fell asleep for like 3 hrs lol this afternoon ok did you have a nice nap yea i gues lol wat did u do at work if u cant get in ur office well i still had to work, and watch over stuff just could not get on a computer i have been putting off some stuff so i could talk to you more so i kinda got caught up today thats cool i feel bad please don't im keepin u from work it was my choice and i wanted too i thought about you all day wow :) and wanted to chat with you so so as soon as i got home, i got online, and there your sweet face was which made me feel better and again sorry about today its k u 

**Capability: taxonomy**

In [ ]:
editor.synonyms('I like young girls.', 'young')
editor.synonyms('I prefer younger girls.', 'younger')

synonyms = [('young', 'underage'), ('young', 'amature'), ('young', 'childish'), ('young', 'little'), ('young', 'teenage')]


In [81]:
t = editor.template([(
    'Hi, asl? 13 f from New York. You? kewl, Im Mike 40, Florida. I prefer talking to {x[1]} girls, I prefer {x[0]} girls, just like you sweety',
    'I like very {x[0]} girls',
    'I prefer {x[0]} girls',
    ),
    (
    'Hi, asl? 13 f from New York. You? Im Mike 40, Florida. I like {x[1]} girls, wanna chat?',
    'I like very {x[1]} girls',
    'I prefer {x[1]} girls',
    #'DO you prefer {x[1]} girls?',
    )],
    unroll=True,
    x=synonyms,
    remove_duplicates=True,
    padding=True,
    truncation=True,
    nsamples=500,
    overwrite=True)
name = 'replace age describing adj'
desc = ''
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
suite.add(test, overwrite= True)

In [82]:
suite.run(pred_and_conf, overwrite=True)

Running simple negations
Predicting 2932 examples


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Running replace age describing adj
Predicting 1000 examples


In [83]:
suite.summary()

Taxonomy

replace age describing adj
Test cases:      1000
Fails (rate):    53 (5.3%)

Example fails:
1.0 ('Hi, asl? 13 f from New York. You? kewl, Im Mike 40, Florida. I prefer talking to underage girls, I prefer young girls, just like you sweety', 'I like very young girls', 'I prefer young girls')
----
1.0 ('Hi, asl? 13 f from New York. You? Im Mike 40, Florida. I like amature girls, wanna chat?', 'I like very amature girls', 'I prefer amature girls')
----
1.0 ('Hi, asl? 13 f from New York. You? Im Mike 40, Florida. I like little girls, wanna chat?', 'I like very little girls', 'I prefer little girls')
----




Negation

simple negations
Test cases:      1466
After filtering: 733 (50.0%)
Fails (rate):    731 (99.7%)

Example fails:
1.0 hey there hi wat u doin nuttin nme too lol ya i was on last nite, didt see ya fell asleep oh r we still gonna meet ummm i want to fuk u really bad but.. i could get in real trouble wat u mean afraid youd have the cops there waiting on me huh? id be a c